# Import Libraries

In [1]:
# !pip install torchmetrics
# !pip install imageio
# !pip install tdqm
# !pip install matplotlib
# !pip install pandas
# !pip install scipy
# !pip install openpyxl
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# !pip install PyWavelets
# !pip install numpy --upgrade
# !pip install scipy
# !pip install pynvml
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [2]:
# DONE
# Import necessary libraries.
import os
import glob
import imageio
import random, shutil
import torch
import torch.nn as nn
from torchmetrics import AUROC, AveragePrecision, ConfusionMatrix
from tqdm.notebook import tqdm
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.cuda.amp import GradScaler, autocast
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as display
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from PIL import Image
import requests
import pandas as pd
import scipy as sp
import pywt
from scipy.signal import cwt
import scipy
import numpy
import numpy as Numpy
import gc

# Handling Data

In [3]:
# Read the Excel file "ML AP Data 2023 Full Series(Jan2).xlsx" and load it into a DataFrame

df = pd.read_excel("ML AP Data 2023 Full Series(Jan2).xlsx")
df.replace(r'^\s*$', np.nan, regex=True)
df = df.T.reset_index()
df.columns = df.iloc[0, :]
df = df.iloc[1:, :304005]
df

,ID,Sex,Diet,Age,Condition,0,0.001,0.002,0.003,0.004,...,303.99,303.991,303.992,303.993,303.994,303.995,303.996,303.997,303.998,303.999
1,F1,Male,NC,24 wks,NaN,130.1223,129.9399,129.6561,129.3926,129.0074,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,F9,Male,NC,24 wks,NaN,130.4467,130.1021,129.6764,129.1899,128.9466,...,151.2245,150.961,150.5961,150.1907,149.7853,149.3596,148.8122,148.346,147.8798,147.4946
3,F10,Male,NC,24 wks,NaN,131.7845,131.4399,131.2778,130.9737,130.7102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,F17,Male,NC,24 wks,NaN,172.6916,173.6038,174.4146,175.1241,175.9552,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,F18,Male,NC,24 wks,NaN,154.5692,154.9138,155.4409,155.7652,156.1098,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,S29,Male,NC,12 wks,NaN,167.2386,167.9279,168.6779,169.286,169.8739,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,S3,Male,HFD,12 wks,NaN,150.5961,150.4339,150.2515,149.988,149.7853,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,S7,Male,HFD,12 wks,NaN,145.731,146.7649,147.6771,148.6501,149.6636,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,S15,Male,HFD,12 wks,NaN,160.4884,159.8194,158.9275,158.3396,157.468,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#Create intermediary dataframes separating data and metadata to facilitate
#further processing
df_data = df.iloc[:, 5:].T.reset_index()
df_data = df_data.rename(columns={0:"Time"})
df_data.Time = pd.to_datetime(df_data.Time, unit="s")
df_data.index = df_data.Time
df_data = df_data.iloc[:, 1:]
df_meta = df.iloc[:, :5].rename(columns={"Condition":"Surgery"})
df_data

,1,2,3,4,5,6,7,8,9,10,...,133,134,135,136,137,138,139,140,141,142
Time,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:00:00.000,130.1223,130.4467,131.7845,172.6916,154.5692,155.522,131.0142,126.5749,148.0217,155.6233,...,136.5482,120.8382,156.1301,160.6708,122.845,167.2386,150.5961,145.731,160.4884,161.1979
1970-01-01 00:00:00.001,129.9399,130.1021,131.4399,173.6038,154.9138,155.2382,130.5075,126.2911,147.7784,156.0085,...,137.2375,120.453,155.8666,159.9005,122.7031,167.9279,150.4339,146.7649,159.8194,161.1168
1970-01-01 00:00:00.002,129.6561,129.6764,131.2778,174.4146,155.4409,154.8733,130.1831,125.9059,147.4946,156.5558,...,138.0888,119.9462,155.6841,159.3532,122.2571,168.6779,150.2515,147.6771,158.9275,160.8938
1970-01-01 00:00:00.003,129.3926,129.1899,130.9737,175.1241,155.7652,154.63,129.7777,125.5613,147.2919,157.0423,...,138.9402,119.6016,155.4814,158.8464,122.0342,169.286,149.988,148.6501,158.3396,160.6505
1970-01-01 00:00:00.004,129.0074,128.9466,130.7102,175.9552,156.1098,154.4476,129.575,125.014,147.1906,157.5896,...,139.7511,119.257,155.3193,158.3802,121.7706,169.8739,149.7853,149.6636,157.468,160.4276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 00:05:03.995,NaN,149.3596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-01-01 00:05:03.996,NaN,148.8122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-01-01 00:05:03.997,NaN,148.346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Create new columns for multiclass classification
df_meta["SexDiet"] = df_meta.Sex + df_meta.Diet
df_meta["AgeDiet"] = df_meta.Age + df_meta.Diet
df_meta["SexAge"] = df_meta.Sex + df_meta.Age
df_meta["SexAgeDiet"] = df_meta.Sex + df_meta.Age + df_meta.Diet
df_meta

,ID,Sex,Diet,Age,Surgery,SexDiet,AgeDiet,SexAge,SexAgeDiet
1,F1,Male,NC,24 wks,NaN,MaleNC,24 wksNC,Male24 wks,Male24 wksNC
2,F9,Male,NC,24 wks,NaN,MaleNC,24 wksNC,Male24 wks,Male24 wksNC
3,F10,Male,NC,24 wks,NaN,MaleNC,24 wksNC,Male24 wks,Male24 wksNC
4,F17,Male,NC,24 wks,NaN,MaleNC,24 wksNC,Male24 wks,Male24 wksNC
5,F18,Male,NC,24 wks,NaN,MaleNC,24 wksNC,Male24 wks,Male24 wksNC
...,...,...,...,...,...,...,...,...,...
138,S29,Male,NC,12 wks,NaN,MaleNC,12 wksNC,Male12 wks,Male12 wksNC
139,S3,Male,HFD,12 wks,NaN,MaleHFD,12 wksHFD,Male12 wks,Male12 wksHFD
140,S7,Male,HFD,12 wks,NaN,MaleHFD,12 wksHFD,Male12 wks,Male12 wksHFD
141,S15,Male,HFD,12 wks,NaN,MaleHFD,12 wksHFD,Male12 wks,Male12 wksHFD


# Create Directories

In [5]:
# @title Create directories
##############
'''
REMEMBER TO CREATE A FOLDER THAT CONTAINS ALL SCALOGRAMS NOT DIVIDED BY CATEGORY
THOSE WILL BE USED FOR SALIENCY-SCALOGRAMS CONCATENATION
'''
################
def CrtDir (grp, col, exp):
    '''
    create the folders required for experimentation
    grp: Classification (e.g. Age)
    col: items inside classification (12 weeks, 24 weeks)
    exp: experiment number (in frequency)
    '''
    files = ["scal", "scal-cropped"]
    main = f'Scalograms2/freq_{exp}_({col})'
    if os.path.exists(main):
        shutil.rmtree(main)
        os.mkdir(main)
    for f in files:
        os.makedirs(os.path.join(main,f'{f}'))
        if f == "scal-cropped":
            os.makedirs(os.path.join(main,f'{f}', 'train'))
            os.makedirs(os.path.join(main,f'{f}', 'test'))
            os.makedirs(os.path.join(main,f'{f}', 'val'))
            os.makedirs(os.path.join(main,f'{f}', 'all_scal'))
            

        for g in grp:
            if f == "scal-cropped":
                path_train = os.path.join(main,f'{f}', 'train',f'{g}')
                path_test = os.path.join(main,f'{f}', 'test',f'{g}')
                path_val = os.path.join(main,f'{f}', 'val',f'{g}')
                os.makedirs(path_train)
                os.makedirs(path_test)
                os.makedirs(path_val)
            main_path = os.path.join(main,f'{f}',f'{g}')
            os. makedirs(main_path)

# Slicing Scalograms

In [6]:
def slice_scal(grp, col, freq, df, exp, w=16.67):
    '''
      Slice timeseries data into bins with a bin size w,
      and compute the scalogram for each bin.
      ------------------------------------
      grp: the values to be classified
      col: name of the variable to be classified
      freq: sampling rate in the timeseries data
      df: dataframe containing timeseries data
      exp: experiment number. Used to name folders
      w: bin size in seconds
      '''
    # Change W to represent the number of points depending on the sampling rate
    w = int(w * freq)

    for g in grp:

        # slice the dataframe to include only the variable being classified
        y = df[(df[col] == g)]

        # Naming convention; names images numerically.
        j = 0

        for i in range(y.shape[0]):
            x = y.iloc[i, 9:]

            for z in range(int(x.shape[0] / w)):

                # if there are not enough points to create a new scalogram with the current window, discard the remaining points
                if x.iloc[int(z * w): int((z + 1) * w)].isna().values.any() == True:
                    break

                # Add zeros to pad the image numbers so that sorting works properly
                if len(str(j)) == 1:
                    h = "000" + str(j)
                elif len(str(j)) == 2:
                    h = "00" + str(j)
                elif len(str(j)) == 3:
                    h = "0" + str(j)
                else:
                    h = str(j)
                
                # the Wavelet Transform computes the inner product of the signal with a scaled  
                # and translated version of the wavelet function over all possible scales and time positions.
                
                # Create a new scalogram with a bin size w and sampling rate freq
                signal_slice = x.iloc[int(z * w): int((z + 1) * w)]
                scales = np.arange(1, 128)  # Choose appropriate scales for the scalogram
                scalogram, frequencies = pywt.cwt(signal_slice, scales, 'morl')

                
                # This results in a two-dimensional representation, where one axis represents time, the other represents scale, 
                # and the value at each point represents the strength of the correlation
                
                # Plot and save the scalogram
                plt.figure(figsize=(6.4, 4.8))
                plt.imshow(np.abs(scalogram), aspect='auto', cmap='viridis')
                plt.colorbar(label='Magnitude')
                plt.xlabel('Time (samples)')
                plt.ylabel('Scale')
                plt.title(f'Scalogram - {col} ({g})')
                plt.savefig(f'Scalograms2/freq_{freq}_({col})/scal/{g}/{g + str(h)}.png')

                # Clear old figure away from memory
                plt.close()
                j = j + 1

# Cropping all Scalograms

In [7]:
# @title Cropping all scalograms
def crop_scal(grp, exp):
    '''
      Crop scalograms to remove all white spaces and axes, leaving behind only the plot behind
      grp: The values being classified. Used here to retrieve images and save them to the appropriate directory
      exp: Experiment number. Used to name folders
    '''
    
      # Box defines the cropping parameters to perform on the plot. Sometimes, the default plot size changes from IDE to another,
      # Should you change your IDE, check a single trial in the cell above to see if the plot is cropped properly
      # The format of the box is [left, upper, right, lower]
      # Example: [left, upper, right, lower] for a 6.4 x 4.8 inch image (aspect ratio 4:3)

    box = [81, 59, 477, 427]
    
    for g in grp:
        path = os.path.join(f'Scalograms2/freq_{exp}_({col})/scal', f'{g}')
        j = 0  
        for filename in os.listdir(path):

            scal  =  os.path.join(path, f"{filename}")

            if len(str(j)) == 1:
                h = "000" + str(j)
            elif len(str(j)) == 2:
                h = "00" + str(j)
            elif len(str(j)) == 3:
                h = "0" + str(j)
            else:
                h = str(j)
                
            #Open image, crop it, and save to the appropriate directory
            im = Image.open(scal)
            region = im.crop(box)

            #Save in the respective group folder
            region.save(f'Scalograms2/freq_{exp}_({col})/scal-cropped/{g}/{g+str(h)}.png')
            #And in the folder containing all scalograms
            region.save(f'Scalograms2/freq_{exp}_({col})/scal-cropped/all_scal/{g+str(h)}.png')
            j = j+1


# Helper Functions

In [8]:
# @title Helper functions (run me)

def set_device():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device != "cuda":
        print("WARNING: For this notebook to perform best, "
              "if possible, in the menu under `Runtime` -> "
              "`Change runtime type.`  select `GPU` ")
    else:
        print("GPU is enabled in this notebook.")

    return device


#  Plotting function.
def plot_loss_accuracy(train_loss, train_acc, validation_loss, validation_acc, freq, col, rn):
    epochs = len(train_loss)
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.plot(list(range(epochs)), train_loss, label='Training Loss')
    ax1.plot(list(range(epochs)), validation_loss, label='Validation Loss')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.set_title('Epoch vs Loss')
    ax1.legend()

    ax2.plot(list(range(epochs)), train_acc, label='Training Accuracy')
    ax2.plot(list(range(epochs)), validation_acc, label='Validation Accuracy')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.set_title('Epoch vs Accuracy')
    ax2.legend()
    fig.set_size_inches(15.5, 5.5)
    fig.savefig(f'Figures/Experiments/Freq_{str(freq)}_{col}_{str(rn)}.png')
    plt.clf()
    return fig

In [9]:
device = set_device()

GPU is enabled in this notebook.


In [10]:
print(f"Is CUDA supported by this system?{torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device:{torch.cuda.current_device()}")
print(f"Name of current CUDA device:{torch.cuda.get_device_name(cuda_id)}")

Is CUDA supported by this system?True
CUDA version: 12.1
ID of current CUDA device:0
Name of current CUDA device:NVIDIA GeForce RTX 3050 Laptop GPU


In [11]:
print(torch.__version__)
print(torch.cuda.is_available())

2.1.0
True


In [12]:
# Reset GPU allocation to 0 to resolve out of memory errors
# Empties CUDA cache to deal with out of memory errors
def report_gpu():
    #print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

# Create folder with training, testing and validation data

In [13]:
# @title Create folder with training, testing and validation data.

def splt_scal(grp, exp):
    '''  
    Create train, test, and validation folders, and shuffle images into them (ratios 8:1:1)
    '''
    scalograms_dir = [f'Scalograms2/freq_{exp}_({col})/scal-cropped']
    folder_names = [f'Scalograms2/freq_{exp}_({col})/scal-cropped/train',
                    f'Scalograms2/freq_{exp}_({col})/scal-cropped/test',
                    f'Scalograms2/freq_{exp}_({col})/scal-cropped/val']


    #Create train test val folders 
    for f in folder_names:
        if os.path.exists(f):
            shutil.rmtree(f)
            os.mkdir(f)
        else:
            os.mkdir(f)


      #Group 1 copying
    for g in grp:
        # find all images & split in train, test, and validation
        src_file_paths= []
        for im in glob.glob(os.path.join(scalograms_dir[0], f'{g}',"*.png"), recursive=True):
            src_file_paths.append(im)
        random.shuffle(src_file_paths)
        #src_file_paths = sorted(src_file_paths) #try sorted data

        train_files = src_file_paths[:int(len(src_file_paths) * 0.8)] #Take 80% of files into training
        val_files = src_file_paths[int(len(src_file_paths) * 0.8):int(len(src_file_paths) * 0.9)] #10% into validation
        test_files = src_file_paths[int(len(src_file_paths) * 0.9):] #And 10% into testing

        #  make destination folders for train and test images
        for f in folder_names:
            if not os.path.exists(os.path.join(f , f"{g}")):
                os.mkdir(os.path.join(f , f"{g}"))

        # copy training and testing images over
        for f in train_files:
            shutil.copy(f, os.path.join(os.path.join(folder_names[0]) + '/', f"{g}", os.path.split(f)[1]))
        for f in test_files:
            shutil.copy(f, os.path.join(os.path.join(folder_names[1]) + '/',f"{g}", os.path.split(f)[1]))
        for f in val_files:
            shutil.copy(f, os.path.join(os.path.join(folder_names[2]) + '/',f"{g}", os.path.split(f)[1]))
    return folder_names

# Data Loading Scalograms

In [14]:
# @title Data loading
def load_scal(folder_names, batch_size = 25, shuffle=True):

    train_dataset = datasets.ImageFolder(
        folder_names[0],
        transforms.Compose([
            transforms.ToTensor(),
        ]))

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=0)

    val_dataset = datasets.ImageFolder(
        folder_names[2],
        transforms.Compose([
            transforms.ToTensor(),
        ]))

    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=0)

    test_dataset = datasets.ImageFolder(
        folder_names[1],
        transforms.Compose([
            transforms.ToTensor(),
        ]))

    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=0)
    return train_loader, val_loader, test_loader

# Make a CNN & train it to predict genres

In [15]:
# @title Make a CNN & train it to predict genres.

class b2b_net(nn.Module):
    def __init__(self):
        """Intitalize neural net layers"""
        
        super(b2b_net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=2, stride=1, padding=0)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=2, stride=1, padding=0)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=2, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=2, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=2, stride=1, padding=1)
#         self.conv6 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=2, stride=1, padding=1)
#         self.conv7 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=2, stride=1, padding=1)
#         self.conv8 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=2, stride=1, padding=1)
#         self.conv9 = nn.Conv2d(in_channels=1024, out_channels=2048, kernel_size=2, stride=1, padding=1)
#         self.conv10 = nn.Conv2d(in_channels=2048, out_channels=4096, kernel_size=2, stride=1, padding=1)
        
        self.fc1 = nn.Linear(in_features=19968, out_features=len(grp))
#         self.fc1 = nn.Linear(in_features=6272, out_features=len(grp))
    
        self.batchnorm1 = nn.BatchNorm2d(num_features=8)
        self.batchnorm2 = nn.BatchNorm2d(num_features=16)
        self.batchnorm3 = nn.BatchNorm2d(num_features=32)
        self.batchnorm4 = nn.BatchNorm2d(num_features=64)
        self.batchnorm5 = nn.BatchNorm2d(num_features=128)
#         self.batchnorm6 = nn.BatchNorm2d(num_features=256)
#         self.batchnorm7 = nn.BatchNorm2d(num_features=512)
#         self.batchnorm8 = nn.BatchNorm2d(num_features=1024)
#         self.batchnorm9 = nn.BatchNorm2d(num_features=2048)
#         self.batchnorm10 = nn.BatchNorm2d(num_features=4096)

        self.dropout = nn.Dropout(p=0.4, inplace=False)

    def forward(self, x):

        # Conv layer 1.
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)

        # Conv layer 2.
        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)

        # Conv layer 3.
        x = self.conv3(x)
        x = self.batchnorm3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)

        # Conv layer 4.
        x = self.conv4(x)
        x = self.batchnorm4(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)

        # Conv layer 5.
        x = self.conv5(x)
        x = self.batchnorm5(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        
#         # Conv layer 6.
#         x = self.conv6(x)
#         x = self.batchnorm6(x)
#         x = F.relu(x)
#         x = F.max_pool2d(x, kernel_size=2)

#         # Conv layer 7.
#         x = self.conv7(x)
#         x = self.batchnorm7(x)
#         x = F.relu(x)
#         x = F.max_pool2d(x, kernel_size=2)
        
#         # Conv layer 8.
#         x = self.conv8(x)
#         x = self.batchnorm8(x)
#         x = F.relu(x)
#         x = F.max_pool2d(x, kernel_size=2)
        
        
#         # Conv layer 9.
#         x = self.conv9(x)
#         x = self.batchnorm9(x)
#         x = F.relu(x)
#         x = F.max_pool2d(x, kernel_size=2)
        
#         # Conv layer 10.
#         x = self.conv10(x)
#         x = self.batchnorm10(x)
#         x = F.relu(x)
#         x = F.max_pool2d(x, kernel_size=2)


        # Fully connected layer 1.
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc1(x)
        x = F.softmax(x, dim = 1)

        return x

# Model Handler

In [16]:
# @title Model Handler

class ModelHandler():
    def __init__(self, model, device, epochs, learning_rate=0.00005):
        self.model = model
        self.device = device
        self.epochs = epochs
        self.criterion =  nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        self.scaler = GradScaler()

    def train(self, train_loader, validation_loader):
        train_loss, validation_loss = [], []
        train_acc, validation_acc = [], []
        #with (range(self.epochs)) as tepochs:
        with tqdm(range(self.epochs), unit='epoch') as tepochs:
            tepochs.set_description('Training')
            for epoch in tepochs:
                self.model.train()

                # keep track of the running loss
                running_loss = 0.
                correct, total = 0, 0

                for data, target in train_loader:
#                   # getting the training set
#                   data, target = data.to(device), target.to(device)
#                   # Get the model output (call the model with the data from this batch)
#                   output = self.model(data)
#                   # Zero the gradients out)
#                   self.optimizer.zero_grad()
#                   # Get the Loss
#                   loss = self.criterion(output, target)
#                   # Calculate the gradients
#                   loss.backward()
#                   # Update the weights (using the training step of the optimizer)
#                   self.optimizer.step()
#                   # Zero the gradients out

                    self.optimizer.zero_grad() 

                    #getting the training set
                    data, target = data.to(device), target.to(device)
                    with autocast(dtype=torch.float16): 
                    # Get the model output (call the model with the data from this batch)
                        output = self.model(data)
                        # Get the Loss
                        loss = self.criterion(output, target)
                  # Calculate the gradients
                    self.scaler.scale(loss).backward()
                  # Update the weights (using the training step of the optimizer)
                    self.scaler.step(self.optimizer)
                    self.scaler.update()

                    tepochs.set_postfix(loss=loss.item())
                    running_loss += loss.item()  # add the loss for this batch

                  # get accuracy
                    _, predicted = torch.max(output, 1)
                    total += target.size(0)
                    correct += (predicted == target).sum().item()

                # append the loss for this epoch
                train_loss.append(running_loss/len(train_loader))
                train_acc.append(correct/total)

                # evaluate on validation data
                self.model.eval()
                running_loss = 0.
                correct, total = 0, 0

                for data, target in validation_loader:
                    # getting the validation set
                    self.optimizer.zero_grad()
                    data, target = data.to(device), target.to(device)

                    with autocast(dtype=torch.float16):
                        output = self.model(data)
                        loss = self.criterion(output, target)
                    tepochs.set_postfix(loss=loss.item())
                    running_loss += loss.item()
                  # get accuracy
                    _, predicted = torch.max(output, 1)
                    total += target.size(0)
                    correct += (predicted == target).sum().item()

                validation_loss.append(running_loss/len(validation_loader))
                validation_acc.append(correct/total)

        return train_loss, train_acc, validation_loss, validation_acc

    def test(self, test_loader):
        test_loss, test_acc = 0, 0
        outputs = torch.tensor([]).to(device)
        targets = torch.tensor([]).to(device)
        # with tqdm(range(self.epochs), unit='epoch') as tepochs:
          # tepochs.set_description('Testing')
          # evaluate on validation data
        self.model.eval()
        running_loss = 0.
        correct, total = 0, 0
        #pre = 0

        for data, target in test_loader:
          # getting the validation set
            data, target = data.to(device), target.to(device)
            self.optimizer.zero_grad()
            with autocast(dtype=torch.float16):
                output = self.model(data)

          # get accuracy
            _, predicted = torch.max(output, 1)
            total += float(target.size(0))
            correct += float((predicted == target).sum().item())
            test_acc = correct/total
            outputs = torch.cat((outputs, output), 0)
            targets = torch.cat((targets, target), 0)

        return outputs, targets, test_acc

    def visualize(self, test_loader, name_dic, freq, grp, rn):
        '''
        Create saliency maps for the scalograms produced
        -------
        test_loader: loads test images into the model. 
                     Needs to be set up with batch size 1, and shuffle = False in the loadscal function
        name_dic: dictionary created using func "create_name_dic" that names saliency maps after their parent scalograms
        -------
        #References:
        #https://towardsdatascience.com/saliency-map-using-pytorch-68270fe45e80
        #https://medium.datadriveninvestor.com/visualizing-neural-networks-using-saliency-maps-in-pytorch-289d8e244ab4
        #https://github.com/sijoonlee/deep_learning/blob/master/cs231n/NetworkVisualization-PyTorch.ipynb
        '''
        main = "Saliency"
        path = os.path.join(main, f"{freq}_{grp}_{rn}")
        if not os.path.exists(main):
            os.mkdir(main)
        if not os.path.exists(path):
            os.mkdir(path)
        
        i = 0 
        for X, target in test_loader:
            X, target = X.to(device), target.to(device)

            # we would run the model in evaluation mode
            model.eval()

            # we need to find the gradient with respect to the input image, so we need to call requires_grad_ on it
            X.requires_grad_()

            scores = model(X)

            scores = (scores.gather(1, target.view(-1, 1)).squeeze())

            #backward pass
            scores.backward()

            saliency, _ = torch.max(X.grad.data.abs(),dim=1)

            # code to plot the saliency map as a heatmap
            plt.imshow(saliency[0].clone().detach().cpu(), cmap=plt.cm.hot)
            plt.axis('off')
            plt.savefig(os.path.join(path, f"{name_dic[i]}"))
            plt.clf()
            del X, target, scores, saliency
            i = i + 1

# Creates dictionary to help with the naming of saliency images

In [17]:
def create_name_dic(freq, col):
    '''
    Creates dicitonary to help with the naming of saliency images.
    Used concurrently with data loader (load scal) setup with batch size 1 and shuffle = False
    in order to produce saliency maps in the same order as test images, and name them accordingly.
    '''
    name_dic = {}
    i = 0
    path = f"Scalograms2/freq_{str(freq)}_({col})/scal-cropped/test"
    for folder in os.listdir(path):
        for file in os.listdir(os.path.join(path, folder)):
            name_dic[i] = filedictionary 
            i += 1
    return name_dic

# Downsampler

In [18]:
#@title Downsampler
def downsample(freq, df = df_data, meta = df_meta):
    '''
    Downsample BP data to specified samplerate in milliseconds by taking the mean
    
    Downsampling is a process of reducing the sampling rate or resolution of a signal or image. 
    It is done to improve efficiency, reduce file size, or enhance performance. 
    It can also be used to decrease the bit rate when transmitting over a limited bandwidth or to convert to a more limited audio format.
    '''
    #convert the samplerate provided (in Hz) to time in milliseconds.
    ms = 1 / freq * 1000
    ms = round(ms, 3)
    #Resample the dataframe using the df.resample method
    df = df.resample(f"{ms}ms").mean()
    df = pd.concat([df_meta, df.reset_index(drop=True).T], axis = 1)
    return df

NameError: name 'df_data' is not defined

# Modeling Cycle

In [23]:
import random

# Generate 10 random numbers from 1 to 1000000 to be used as random seeds for the model
# used to ensure that results are reproducible
# In other words, using this parameter makes sure that anyone who re-runs your code will get the exact same outputs.
#randomlist = np.random.randint(1, 1000000, (10,))
randomlist = [200396, 544352, 524927, 635473, 609072, 680561, 227999, 156037, 535490, 208398]

#Umcomment this to create an Experiments folder
# os.makedirs('Experiments')
exp = 1

# sampling rate in the timeseries data
freqs = [140, 120, 100, 80, 70, 60, 50, 40]

In [20]:
#Save results in a dicitonary

multiclassresults = {}
multiclassresults = pd.DataFrame()

In [25]:
exp = 1
grps = {
        'Age': ['12 wks', '24 wks'],
        'Sex': ['Male', 'Female'],
        'Diet': ['HFD', 'NC'],
        'SexDiet': ['MaleNC', 'FemaleNC', 'MaleHFD', 'FemaleHFD'],
        'AgeDiet': ['24 wksNC', '24 wksHFD', '12 wksNC', '12 wksHFD'],
        'SexAge': ['Male12 wks', 'Male24 wks', 'Female12 wks', 'Female24 wks'],
        'SexAgeDiet': ['Male24 wksNC', 'Male24 wksHFD', 'Male12 wksNC', 'Male12 wksHFD',
                       'Female24 wksNC', 'Female24 wksHFD', 'Female12 wksNC', 'Female12 wksHFD']
       }

In [26]:
for col, grp in grps.items():
    for freq in tqdm(freqs):
        for rn in randomlist:
            # Load your pre-trained model
            # net = b2b_net().to(device)
            model = b2b_net().to(device)  # Create an instance of your model class with GPU
            model.load_state_dict(torch.load(f"Models/{freq}_{col}_{rn}"))  # Load the saved parameters
            model.eval()  # Set the model to evaluation mode
            # net.load_state_dict(torch.load(f"Models/{freq}_{col}_{rn}"))

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

In [58]:
import gradio as gr
import numpy as np
from PIL import Image
import tempfile
import torch


# Define preprocess_input function
def preprocess_input(input_scalogram):
    # Load the image using PIL
    pil_image = Image.open(input_scalogram)
    
    # Convert the image to the format expected by your model (e.g., RGB or grayscale)
    pil_image = pil_image.convert("RGB")  # Adjust based on your requirements
    
    # Resize the image if needed
    pil_image = pil_image.resize((224, 224))  # Adjust dimensions
    
    # Convert PIL image to a numpy array
    input_array = np.asarray(pil_image)
    
    # Flatten the input array and normalize
    input_array = input_array.flatten() / 255.0
    
    # Return the processed input
    return input_array

# Define postprocess_output function
def postprocess_output(output):
    # Assuming output is a tuple (predicted_class, confidence)
    predicted_class, confidence = output
    return f"Predicted Class: {predicted_class}, Confidence: {confidence}"

def predict_scalogram(input_scalogram):
    if isinstance(input_scalogram, str):
        # If it's a string, assume it's a file path
        input_scalogram_path = input_scalogram
    else:
        # If it's not a string, assume it's a NumPy array
        # You might need to convert it to an image format compatible with your model
        input_scalogram = np.array(input_scalogram)
        pil_image = Image.fromarray(input_scalogram)
        with tempfile.NamedTemporaryFile(delete=False) as temp_file:
            pil_image.save(temp_file, format='PNG')
            input_scalogram_path = temp_file.name

    # Preprocess the input
    processed_input = preprocess_input(input_scalogram_path)

    # Convert the input to a PyTorch tensor and move to the same device as the model
    input_tensor = torch.from_numpy(processed_input).unsqueeze(0).permute(0, 3, 1, 2).float()
    input_tensor = input_tensor.to(device)  # Assuming device is defined

    # Use the model for prediction
    result = model(input_tensor)

    # Postprocess the output
    processed_output = postprocess_output(result)

    # For demonstration, assuming a tuple (predicted_class, confidence) as output
    return processed_output



# Define Gradio Interface
iface = gr.Interface(
    fn=predict_scalogram,
    inputs="image",  # Adjust according to your input type
    outputs="text",  # Adjust according to your output type
    live=True,
    title="Scalogram Prediction Demo",
    description="Upload a scalogram for prediction.",
    capture_session=True
)

iface.launch()


C:\Users\ahmed\AppData\Local\Temp\ipykernel_2236\1695168632.py:66: GradioDeprecationWarning: `capture_session` parameter is deprecated, and it has no effect
  iface = gr.Interface(


Running on local URL:  http://127.0.0.1:7891

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\ahmed\anaconda3\envs\clone\Lib\site-packages\gradio\routes.py", line 534, in predict
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ahmed\anaconda3\envs\clone\Lib\site-packages\gradio\route_utils.py", line 226, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ahmed\anaconda3\envs\clone\Lib\site-packages\gradio\blocks.py", line 1554, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ahmed\anaconda3\envs\clone\Lib\site-packages\gradio\blocks.py", line 1192, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ahmed\anaconda3\envs\clone\Lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thre

In [28]:
# One modeling cycle

for col, grp in grps.items():
    
    #Uncomment this to work on freq = 100 only for the saliency experiments
#     freqs = [100]
    for freq in tqdm(freqs):
        
#         df_done = downsample(freq)
#         print("\033[92mDownsampler is done!")  
#         CrtDir(grp = grp, col = col, exp = freq)
#         print("\033[92mCreating directories is done!")
#         slice_scal(freq = freq, df = df_done, w = 16.67, grp = grp, col = col, exp = freq)
#         print("\033[92mSlicing scalograms is done!")
#         crop_scal(grp = grp, exp = freq)
#         print("\033[92mCropping scalograms is done!")
    
        for rn in randomlist:
            print(freq)

            random.seed(rn)
            torch.manual_seed(rn)
            torch.cuda.manual_seed_all(rn)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
            np.random.seed(rn)
            os.environ['PYTHONHASHSEED'] = str(rn)

            folder_names = splt_scal(grp = grp, exp = freq)
            print("\033[92mTrain val test splitting is done!")
            
            
            train_loss, train_acc = [], []
            validation_loss, validation_acc = [], []
            
            report_gpu() #Empties CUDA cache to deal with out of memory errors
            
            # Creating Model
            net = b2b_net().to(device)
            # net.load_state_dict(torch.load(f"Models/{freq}_{col}_{rn}"))
            handler = ModelHandler(net, device, 30, 0.00005)
            
            train_loader, val_loader, test_loader = load_scal(folder_names)

            train_loss, train_acc, validation_loss, validation_acc = handler.train(train_loader, val_loader)
            print("\033[92mTraining is done!")

            # Test Phase
            test_acc = 0
            output, target, test_acc = handler.test(test_loader)
            print("\033[92mTesting is done!")

            # # To CPU
            # train_loss, train_acc = torch.tensor(train_loss).cpu(), torch.tensor(train_acc).cpu()
            # validation_loss, validation_acc = torch.tensor(validation_loss).cpu(), torch.tensor(validation_acc).cpu()
            
            output, target = output.detach().clone().cpu(), target.detach().clone().cpu()
            conf_mat = ConfusionMatrix(task="multiclass", num_classes = len(grp))
            conf_mat = conf_mat(output, target)
            # print(conf_mat)
            TP, FN, FP, TN = conf_mat[0][0], conf_mat[0][1], conf_mat[1][0], conf_mat[1][1]
            
            #Calculate some performance metrics
            specificity = TN / (TN + FP)
            sensitivity = TP / (TP + FN)
            ppv         = TP / (TP + FP)
            npv         = TN / (TN + FN)
            
            auroc = AUROC(task="multiclass", num_classes = len(grp))
            auprc = AveragePrecision(task="multiclass", num_classes = len(grp))
            multiclassresults[str(freq) + "_" + col + "_" + str(rn)] = [round(test_acc * 100, 2), 
                                                              auroc(output, target.type(torch.int64)),
                                                              auprc(output, target.type(torch.int64)),
                                                              specificity,
                                                              sensitivity,
                                                              ppv,
                                                              npv                                                            
            ]
            
            print(test_acc)

            exp += 1
            
            
            #Plot Epoch vs Accuracy & Epoch vs Loss
            plot_loss_accuracy(train_loss, train_acc, validation_loss, validation_acc, freq, col, rn)
            
            #Create the "Models" directory if it doesn't exist
            if not os.path.exists("Models"):
                os.makedirs("Models")
                
            #Save model weights and results    
            torch.save(net.state_dict(), f"Models/{freq}_{col}_{rn}")
            multiclassresults.to_csv("multiclassresults.csv")
            del net
            print(f"\033[92mExp {exp - 1} is finally done!")
            

  0%|          | 0/1 [00:00<?, ?it/s]

100
Train val test splitting is done!


  0%|          | 0/30 [00:00<?, ?epoch/s]

Training is done!
Testing is done!
0.9801587301587301
Exp 1 is finally done!
100
Train val test splitting is done!


  0%|          | 0/30 [00:00<?, ?epoch/s]

...

KeyboardInterrupt: 

<Figure size 1550x550 with 0 Axes>

<Figure size 1550x550 with 0 Axes>

<Figure size 1550x550 with 0 Axes>

<Figure size 1550x550 with 0 Axes>

In [1]:
df_done

NameError: name 'df_done' is not defined

In [5]:
result = df = pd.read_csv("multiclassresults.csv")

In [6]:
result

,Unnamed: 0,140_Age_200396,140_Age_544352,140_Age_524927,140_Age_635473,140_Age_609072,140_Age_680561,140_Age_227999,140_Age_156037,140_Age_535490
0,0,96.84,89.33,96.44,93.28,96.05,94.47,94.86,97.23,96.05
1,1,tensor(0.9614),tensor(0.8871),tensor(0.9515),tensor(0.9485),tensor(0.9667),tensor(0.9514),tensor(0.9371),tensor(0.9625),tensor(0.9521)
2,2,tensor(0.9545),tensor(0.8713),tensor(0.9333),tensor(0.9396),tensor(0.9565),tensor(0.9370),tensor(0.9138),tensor(0.9476),tensor(0.9329)
3,3,tensor(0.9649),tensor(0.8509),tensor(0.9386),tensor(0.8860),tensor(0.9386),tensor(0.9298),tensor(0.9386),tensor(0.9561),tensor(0.9298)
4,4,tensor(0.9712),tensor(0.9281),tensor(0.9856),tensor(0.9712),tensor(0.9784),tensor(0.9568),tensor(0.9568),tensor(0.9856),tensor(0.9856)
5,5,tensor(0.9712),tensor(0.8836),tensor(0.9514),tensor(0.9122),tensor(0.9510),tensor(0.9433),tensor(0.9500),tensor(0.9648),tensor(0.9448)
6,6,tensor(0.9649),tensor(0.9065),tensor(0.9817),tensor(0.9619),tensor(0.9727),tensor(0.9464),tensor(0.9469),tensor(0.9820),tensor(0.9815)


In [7]:
res = pd.DataFrame(result).T.reset_index()
res.columns = ['Frequency', 'Test_Accuracy', 'AUROC', 'AUPRC','Specificity', "Sensitivity", "PPV", "NPV"]
res = res.drop(0)
res = res.iloc[0:].reset_index(drop=True)
res["Frequency"], res["Classification"], res['RN'] = ([c.split("_")[0] for c in res.Frequency],
                                                      [d.split("_")[1] for d in  res.Frequency], 
                                                      [e.split("_")[2] for e in res.Frequency])
    

res['AUROC'], res['AUPRC'], res['Specificity'], res["Sensitivity"], res["PPV"], res["NPV"] = ([float(str(c).split("(")[1][:-1]) for c in res['AUROC']],
                                                                                [float(str(c).split("(")[1][:-1]) for c in res['AUPRC']],
                                                                                [float(str(c).split("(")[1][:-1]) for c in res['Specificity']],
                                                                                [float(str(c).split("(")[1][:-1]) for c in res['Sensitivity']],
                                                                                [float(str(c).split("(")[1][:-1]) for c in res['PPV']], 
                                                                                [float(str(c).split("(")[1][:-1]) for c in res['NPV']])

res.head()

,Frequency,Test_Accuracy,AUROC,AUPRC,Specificity,Sensitivity,PPV,NPV,Classification,RN
0,140,96.84,0.9614,0.9545,0.9649,0.9712,0.9712,0.9649,Age,200396
1,140,89.33,0.8871,0.8713,0.8509,0.9281,0.8836,0.9065,Age,544352
2,140,96.44,0.9515,0.9333,0.9386,0.9856,0.9514,0.9817,Age,524927
3,140,93.28,0.9485,0.9396,0.8860,0.9712,0.9122,0.9619,Age,635473
4,140,96.05,0.9667,0.9565,0.9386,0.9784,0.9510,0.9727,Age,609072


In [8]:
res[0:50]

,Frequency,Test_Accuracy,AUROC,AUPRC,Specificity,Sensitivity,PPV,NPV,Classification,RN
0,140,96.84,0.9614,0.9545,0.9649,0.9712,0.9712,0.9649,Age,200396
1,140,89.33,0.8871,0.8713,0.8509,0.9281,0.8836,0.9065,Age,544352
2,140,96.44,0.9515,0.9333,0.9386,0.9856,0.9514,0.9817,Age,524927
3,140,93.28,0.9485,0.9396,0.8860,0.9712,0.9122,0.9619,Age,635473
4,140,96.05,0.9667,0.9565,0.9386,0.9784,0.9510,0.9727,Age,609072
5,140,94.47,0.9514,0.9370,0.9298,0.9568,0.9433,0.9464,Age,680561
6,140,94.86,0.9371,0.9138,0.9386,0.9568,0.9500,0.9469,Age,227999
7,140,97.23,0.9625,0.9476,0.9561,0.9856,0.9648,0.9820,Age,156037
8,140,96.05,0.9521,0.9329,0.9298,0.9856,0.9448,0.9815,Age,535490


In [10]:
i = 0
columns = [
           'Frequency', 'Classification',
#            'AUROC mean', 'AUROC CI-upper', 'AUROC CI lower',
#            'AUPRC mean', 'AUPRC CI-upper', 'AUPRC CI lower',
           'Specificity mean', 'Specificity CI-upper', 'Specificity CI-lower',
           'Sensitivity mean', 'Sensitivity CI-upper', 'Sensitivity CI-lower',
           'PPV mean', 'PPV CI-upper', 'PPV CI-lower',
           'NPV mean', 'NPV CI-upper', 'NPV CI-lower']

resavg = pd.DataFrame(columns = columns)
for freq in freqs:
    for col, grp in grps.items():
        resavg.loc[i, 'Frequency'] = freq
        resavg.loc[i, 'Classification'] = col
        
#         x = res[(res['Frequency'] == str(freq)) & (res['Classification'] == col)].loc[:, 'AUROC']
#         resavg.loc[i, 'AUROC mean'] = x.mean() * 100
#         resavg.loc[i, 'AUROC CI-upper'], resavg.loc[i, 'AUROC CI-lower'] = sp.stats.t.interval(confidence=0.95,
#                                                                                                 df=len(x)-1, loc=np.mean(x), scale=sp.stats.sem(x))
#         resavg.loc[i, 'AUROC CI-upper'], resavg.loc[i, 'AUROC CI-lower'] = (resavg.loc[i, 'AUROC CI-upper'] * 100,
#                                                                                         resavg.loc[i, 'AUROC CI-lower'] * 100)
        
#         x = res[(res['Frequency'] == str(freq)) & (res['Classification'] == col)].loc[:, 'AUPRC']
#         resavg.loc[i, 'AUPRC mean'] = x.mean() * 100
#         resavg.loc[i, 'AUPRC CI-upper'], resavg.loc[i, 'AUPRC CI-lower'] = sp.stats.t.interval(confidence=0.95,
#                                                                                                df=len(x)-1, loc=np.mean(x), scale=sp.stats.sem(x))
#         resavg.loc[i, 'AUPRC CI-upper'], resavg.loc[i, 'AUPRC CI-lower'] = (resavg.loc[i, 'AUPRC CI-upper'] * 100,
#                                                                                         resavg.loc[i, 'AUPRC CI-lower'] * 100)

        
        x = res[(res['Frequency'] == str(freq)) & (res['Classification'] == col)].loc[:, 'Specificity']
        resavg.loc[i, 'Specificity mean'] = x.mean() * 100
        resavg.loc[i, 'Specificity CI-upper'], resavg.loc[i, 'Specificity CI-lower'] = sp.stats.t.interval(confidence=0.95,
                                                                                                df=len(x)-1, loc=np.mean(x), scale=sp.stats.sem(x))
        resavg.loc[i, 'Specificity CI-upper'], resavg.loc[i, 'Specificity CI-lower'] = (resavg.loc[i, 'Specificity CI-upper'] * 100,
                                                                                        resavg.loc[i, 'Specificity CI-lower'] * 100)
        
        
        x = res[(res['Frequency'] == str(freq)) & (res['Classification'] == col)].loc[:, 'Sensitivity']
        resavg.loc[i, 'Sensitivity mean'] = x.mean() * 100
        resavg.loc[i, 'Sensitivity CI-upper'], resavg.loc[i, 'Sensitivity CI-lower'] = sp.stats.t.interval(confidence=0.95,
                                                                                                df=len(x)-1, loc=np.mean(x), scale=sp.stats.sem(x))
        resavg.loc[i, 'Sensitivity CI-upper'], resavg.loc[i, 'Sensitivity CI-lower'] = (resavg.loc[i, 'Sensitivity CI-upper'] * 100,
                                                                                        resavg.loc[i, 'Sensitivity CI-lower'] * 100)
        
        x = res[(res['Frequency'] == str(freq)) & (res['Classification'] == col)].loc[:, 'PPV']
        resavg.loc[i, 'PPV mean'] = x.mean() * 100
        resavg.loc[i, 'PPV CI-upper'], resavg.loc[i, 'PPV CI-lower'] = sp.stats.t.interval(confidence=0.95,
                                                                                                df=len(x)-1, loc=np.mean(x), scale=sp.stats.sem(x))
        resavg.loc[i, 'PPV CI-upper'], resavg.loc[i, 'PPV CI-lower'] = (resavg.loc[i, 'PPV CI-upper'] * 100,
                                                                        resavg.loc[i, 'PPV CI-lower'] * 100)
        
        
        x = res[(res['Frequency'] == str(freq)) & (res['Classification'] == col)].loc[:, 'NPV']
        resavg.loc[i, 'NPV mean'] = x.mean() * 100
        resavg.loc[i, 'NPV CI-upper'], resavg.loc[i, 'NPV CI-lower'] = sp.stats.t.interval(confidence=0.95,
                                                                                                df=len(x)-1, loc=np.mean(x), scale=sp.stats.sem(x))
        resavg.loc[i, 'NPV CI-upper'], resavg.loc[i, 'NPV CI-lower'] = (resavg.loc[i, 'NPV CI-upper'] * 100,
                                                                        resavg.loc[i, 'NPV CI-lower'] * 100)
        
        i += 1
resavg

NameError: name 'freqs' is not defined

In [ ]:
# Create the Figures directory if it does not exist
if not os.path.exists("Figures"):
    os.mkdir("Figures")


def plot_line(metric):
    plt.plot( x['Frequency'].astype('float'), x[f'{metric} mean'], label = f'{metric}')
    plt.fill_between(x['Frequency'].astype('float'), 
                     x[f'{metric} CI-lower'].astype('float'), x[f'{metric} CI-upper'].astype('float'), alpha=.1)

for col in ['Age', 'Sex', 'Diet', 'SexDiet', 'AgeDiet', 'SexAge', 'SexAgeDiet']:
    
    x = resavg[resavg.Classification == col]
    plot_line("Sensitivity")
    plot_line("Specificity")
    plot_line("PPV")
    plot_line("NPV")
#     plt.plot( x['Frequency'].astype('float'), x['AUROC mean'], label = 'AUROC')
#     plt.fill_between(x['Frequency'].astype('float'), 
#                      x['AUROC CI-lower'].astype('float'), x['AUROC CI-upper'].astype('float'), alpha=.1)
    
#     plt.plot( x['Frequency'].astype('float'), x['AUPRC mean'], label = 'AUPRC')
#     plt.fill_between(x['Frequency'].astype('float'), 
#                      x['AUPRC CI-lower'].astype('float'), x['AUPRC CI-upper'].astype('float'), alpha=.1)

    plt.legend()
    plt.xlabel("Frequency")
    plt.title(f"{col} classification by increasing sample rate")
    plt.savefig(f"Figures\class_{col}.png")
    plt.clf();

In [ ]:
##Generating saliency maps for each classification at each freq

freq_list = [140, 120, 100, 80, 70, 60, 50, 40]
for freq in freq_list:
    for col, grp in grps.items():
        for rn in randomlist:
            folder_names = splt_scal(grp = grp, exp = freq)
            model = b2b_net().to(device)
            model.load_state_dict(torch.load(f"Models/{freq}_{col}_{rn}"))
            handler = ModelHandler(model, device, 30, 0.00005)

            _, _, test_loader = load_scal(folder_names, batch_size=1, shuffle=False)
            name_dic = create_name_dic(freq=freq, col=col)
            handler.visualize(test_loader, name_dic, freq, col, rn)
            print(f"\033[92m{col} visualization is done!")

In [ ]:
#Crop the produced saliency maps to remove all white spaces...
# Example: [left, upper, right, lower] for a 6.4 x 4.8 inch image (aspect ratio 4:3)

box = [129, 58, 527, 427]

sal = "Saliency"

freq_list = [140, 120, 100, 80, 70, 60, 50, 40]
for freq in freq_list:
    for col, _ in tqdm(grps.items()):
        for rn in randomlist:
            folder = os.path.join(sal, f"{freq}_{col}_{rn}")
            for file in os.listdir(folder):
                img1 = Image.open(os.path.join(folder, file))
                img1 = img1.crop(box)
                img1.save(os.path.join(folder, file))

In [ ]:
#Fn that concatenates two images together
def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst


#Create new folders that contain saliency-scalogram pairs
##UPDATE THIS CODE TO REFLECT THE PRESENCE OF 10 FOLDERS PER CLASSIFICATION
##ALSO REMEMBER TO ADD THE ALL_SCAL FOLDER THAN CONTAINS ALL SCALOGRAMS IN A GIVEN CLASSIFICATION

# for i in range(len(os.listdir("Saliency")) - 1):
#     main = "Saliency-scalograms"
#     path = os.path.join(main, os.listdir("Saliency")[i])
#     if not os.path.exists(main):
#         os.mkdir(main)
#     if not os.path.exists(path):
#         os.mkdir(path)

##############
'''
REMEMBER TO COPY ALL SCALOGRAMS PER CLASSIFICATION INTO THE FOLDER THAT WILL CONTAINS ALL SCALOGRAMS
THOSE WILL BE USED FOR SALIENCY-SCALOGRAM CONCATENATION
'''
################

In [ ]:
#CONCATENATE SCALOGRAMS WITH THEIR RESPECTIVE SALIENCY MAPS

freq_list = [140, 120, 100, 80, 70, 60, 50, 40]
base_path = "D:/Saliency-scalograms"  # Change the base path to the new directory

for freq in freq_list:
    for col, _ in grps.items():
        scal = os.path.join("Scalograms2", f"freq_{freq}_({col})", "scal-cropped", "all_scal")
        for rn in randomlist:
            sal = f"Saliency/{freq}_{col}_{rn}"
            salscal = os.path.join(base_path, f"{freq}_{col}_{rn}")  # Updated salscal path

            # Create salscal if it is not available
            if not os.path.exists(salscal):
                os.makedirs(salscal)
            for j in range(len(os.listdir(sal))):
                img1 = Image.open(os.path.join(sal, os.listdir(sal)[j]))
                img2 = Image.open(os.path.join(scal, os.listdir(sal)[j]))
                get_concat_h(img1, img2).save(os.path.join(salscal, os.listdir(sal)[j]))

In [ ]:
avg_salscal = "Figures/avg_salscal"

#Create a new folder to save average saliency-scalogram pairs for each model
if not os.path.exists(avg_salscal):
            os.mkdir(avg_salscal)

base_path = "D:/Saliency-scalograms" 

freq_list = [140, 120, 100, 80, 70, 60, 50, 40]
for freq in freq_list:
    for col, _ in grps.items():
        for rn in randomlist:
            salscal = os.path.join(base_path, f"{freq}_{col}_{rn}")
            avg_array = np.zeros((369, 794, 3))
            for file in os.listdir(salscal):
                img_path = os.path.join(salscal, file)
                img = Image.open(img_path)
                array = np.array(img)
                avg_array += array
            avg_array = (avg_array/len(os.listdir(salscal))).astype("uint8")
            avg_image = Image.fromarray(avg_array)
            avg_image.save(os.path.join(avg_salscal, f"{freq}_{col}_{rn}_avg_sal.png"))

In [ ]:
columns = ['Classification', 'Rn'] + list(range(369))
# index = list(np.arange)
results_sal = pd.DataFrame(columns = columns)

In [ ]:
j = 0

freq_list = [140, 120, 100, 80, 70, 60, 50, 40]

for freq in freq_list:
    for col, _ in grps.items():
        for rn in randomlist:    
            sal = f"Saliency/{freq}_{col}_{rn}"    
            avg_freq = np.zeros((369, ))
            for file in os.listdir(sal):
                img_path = os.path.join(sal, file)
                img = Image.open(img_path)
                array = np.array(img).reshape(369, 1592)
                for i in range(1592):    
                    avg_freq += array[:, i]
            denominator = len(os.listdir(sal)) * 1592
            avg_freq = (avg_freq / denominator)

            #Store the average saliency results for each model
            results_sal.loc[j, 'Classification'] = col
            results_sal.loc[j, 'Rn'] = rn
            results_sal.loc[j, list(range(369))] = avg_freq[::-1]
            j += 1

In [ ]:
results_sal

In [ ]:
#Calculate average saliency of models per classification

j = 30
for col, _                        in grps.items():
        x = results_sal[(results_sal['Classification'] == col) &
                        (results_sal['Rn'].isin(randomlist))].iloc[:, 2:]
        #Create a row to store the mean saliency of classification, and calculate it
        results_sal.loc[j, "Classification"] = col
        results_sal.loc[j, "Rn"] = "mean"
        results_sal.loc[j, np.arange(369)] = x.mean(axis=0)
        j += 1
        #Create two rows to store CI-upper and lower
        results_sal.loc[j, "Classification"] = col
        results_sal.loc[j, "Rn"] = "CI-upper"
        j += 1
        results_sal.loc[j, "Classification"] = col
        results_sal.loc[j, "Rn"] = "CI-lower"
        #Calculate the confidence intervals for each classification
        for i in range(369):
            x_column = x.iloc[:30, i]
            # print(x_column)
            results_sal.iloc[j - 1, 2 + i], results_sal.iloc[j, 2 + i] = sp.stats.t.interval(confidence=0.95,
                                                                                 df=len(x_column)-1, loc=np.mean(x_column), scale=sp.stats.sem(x_column))
        j += 1

In [ ]:
for col in ['Age', 'Sex', 'Diet', 'SexDiet', 'AgeDiet', 'SexAge', 'SexAgeDiet']:
    
    x = results_sal[results_sal.Classification == col]
    plt.plot(np.linspace(0, 50, 369), x[x.Rn == 'mean'].iloc[:, 2:].to_numpy(float).flatten(), label = col)
    plt.fill_between(np.linspace(0, 50, 369),
                     x[x.Rn == 'CI-lower'].iloc[:, 2:].to_numpy(float).flatten(),
                     x[x.Rn == 'CI-upper'].iloc[:, 2:].to_numpy(float).flatten(), alpha=.1)
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Time averaged Saliency (a.u.)")
    plt.legend(title="Classification")
    plt.savefig(f"Figures/Time-averaged {col} Saliency.png")
    plt.clf();

In [ ]:
for col in ['Age', 'Sex', 'Diet', 'SexDiet', 'AgeDiet', 'SexAge', 'SexAgeDiet']:
    
    x = results_sal[results_sal.Classification == col]
    mean = sp.signal.resample(x[x.Rn == 'mean'].iloc[:, 4:].to_numpy(float).flatten(), 50)
    up_ci = sp.signal.resample(x[x.Rn == 'CI-upper'].iloc[:, 4:].to_numpy(float).flatten(), 50)
    low_ci = sp.signal.resample(x[x.Rn == 'CI-lower'].iloc[:, 4:].to_numpy(float).flatten(), 50)
    
    plt.plot(np.arange(1, 51), mean)
    mean_max, _ = sp.signal.find_peaks(mean, distance=4, prominence = .35)
    mean_max = mean_max[:5]
    plt.scatter(mean_max + 1, mean[mean_max], s=50, marker='o', facecolor='none', edgecolor='red', label = mean_max )
    
    plt.fill_between(np.arange(1, 51), low_ci, up_ci, alpha=.1)
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Time averaged Saliency (a.u.)")
    plt.legend(title="Frequency peaks")
    plt.title(f"{col} saliency")
#     plt.savefig(f"Figures/Time-averaged {col} Saliency resampled.png")
    plt.show()
    plt.clf()

In [ ]:
#Print top 5 mean frequencies whose saliency was highest for each classification
for col in ['Age', 'Sex', 'Diet', 'SexDiet', 'AgeDiet', 'SexAge', 'SexAgeDiet']:
    x = results_sal[results_sal.Classification == col]
    x = sp.signal.resample(x[x.Rn == 'mean'].iloc[:, 2:].to_numpy(float).flatten(), 50)
    #Print indices of top 5 saliencies, which correspond to frequencies when divided by 50
    print(f"Top 5 {col} frequencies with highest mean time-averaged saliencies", np.argsort(-x)[:5])

In [ ]:
#Create an empty figure for editing purposes

fig = plt.gcf()
fig.set_size_inches(12, 5)
plt.ylabel("Frequency")
plt.yticks(np.arange(0, 51, 10))
plt.show
plt.savefig("blank.png");

In [ ]:
# @title Single image cropping trial

img_path = 'Scalograms2/freq_60_(Age)/scal/12 wks/12 wks0000.png'
image = Image.open(img_path)
image

box = [81, 59, 477, 427] #previously, 3rd number was 390
region = image.crop(box)
region

In [ ]:
#Creating a dummy spectrogram with a colorbar for editing purposes

fig, ax = plt.subplots()
cmap = plt.get_cmap('viridis')
# if x.iloc[i * w:(i + 1)* w].isna().values.any() == False:
pxx,  freq, t, cax = plt.specgram(x[:1000], Fs=50)
fig.colorbar(cax)
plt.savefig('samplespec.png', dpi = 300);

In [ ]:
a, b = np.arange(1, 100), np.arange(1, 100)

plt.plot(a * b[::-1], a / b[::-1]);